In [4]:
# examine_wpr.py

from sqlalchemy import create_engine, text
import pandas as pd

# Adjust this path if your combined.db lives elsewhere
DB_URI = "sqlite:///app/data/combined.db"

def inspect_wpr(table_name: str, engine):
    print(f"\n=== {table_name} ===")
    # 1) Load via a Connection + raw SQL
    with engine.connect() as conn:
        df = pd.read_sql_query(
            text(f"SELECT * FROM {table_name}"),
            conn
        )

    # 2) Find the date‑like column
    date_col = next((c for c in df.columns if c.lower() in ("date","index")), None)
    if date_col is None:
        print(f"  ERROR: no date/index column found (cols={df.columns.tolist()})")
        return

    # 3) Parse and set as index
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col).sort_index()
    df.index.name = "Date"

    # 4) Print summary
    print(f"Shape      : {df.shape[0]} rows × {df.shape[1]} cols")
    print(f"The number of null values is: {df.isnull().sum().sum()}")
    print(f"Date range : {df.index.min().date()} → {df.index.max().date()}")
    print(f"Columns    : {', '.join(df.columns)}")

    # 5) Peek
    print("\nFirst 3 rows:")
    print(df.head(3).to_string())

    print("\nLast  3 rows:")
    print(df.tail(3).to_string())

if __name__ == "__main__":
    engine = create_engine(DB_URI)

    for tbl in ("wpr_truth", "wpr_sliding"):
        inspect_wpr(tbl, engine)


=== wpr_truth ===
Shape      : 1258 rows × 81 cols
The number of null values is: 96
Date range : 2021-11-08 → 2025-04-18
Columns    : EIA CUSHING- OK CRUDE EXCL SPR STK, EIA US CRUDE EXCL SPR STK, EIA US CRUDE STK, EIA PADD1 CRUDE EXCL SPR STK, EIA PADD5 CRUDE EXCL SPR STK, EIA US CRUDE STK IN SPR, EIA PADD2 CRUDE EXCL SPR STK, EIA PADD3 CRUDE EXCL SPR STK, EIA US CRUDE/PETRO EXCL SPR STK, EIA US CRUDE STK AND PETROLEUM PRODUCTS, EIA PADD4 CRUDE EXCL SPR STK, EIA US CRUDE OIL PRODUCTION, EIA US GROSS INPUTS REFINERIES, EIA PADD1 GROSS INPUTS REFINERIES, EIA PADD2 GROSS INPUTS REFINERIES, EIA PADD3 GROSS INPUTS REFINERIES, EIA PADD4 GROSS INPUTS REFINERIES, EIA PADD5 GROSS INPUTS REFINERIES, EIA PADD1 CRUDE DISTILLATION CAP, EIA PADD5 CRUDE DISTILLATION CAP, EIA PADD2 CRUDE DISTILLATION CAP, EIA PADD3 CRUDE DISTILLATION CAP, EIA PADD4 CRUDE DISTILLATION CAP, EIA US EXPORTS OF CRUDE OIL, EIA US PROPANE/PROPYLENE STK, EIA US GAS STK, EIA PADD1 GAS STK, EIA PADD2 GAS STK, EIA PADD3 GAS ST

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

# Adjust this path if your combined.db lives elsewhere
DB_URI = "sqlite:///app/data/combined.db"

def inspect_wpr(table_name: str, engine):
    print(f"\n=== {table_name} ===")
    # 1) Load via a Connection + raw SQL
    with engine.connect() as conn:
        df = pd.read_sql_query(
            text(f"SELECT * FROM {table_name}"),
            conn
        )

    # 2) Find the date‑like column
    date_col = next((c for c in df.columns if c.lower() in ("date","index")), None)
    if date_col is None:
        print(f"  ERROR: no date/index column found (cols={df.columns.tolist()})")
        return

    # 3) Parse and set as index
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col).sort_index()
    df.index.name = "Date"

    # 4) Print summary
    print(f"Shape      : {df.shape[0]} rows × {df.shape[1]} cols")
    print(f"The number of null values is: {df.isnull().sum().sum()}")
    print(f"Date range : {df.index.min().date()} → {df.index.max().date()}")
    print(f"Columns    : {', '.join(df.columns)}")

    # 5) Peek
    print("\nFirst 3 rows:")
    print(df.head(3).to_string())

    print("\nLast  3 rows:")
    print(df.tail(3).to_string())

if __name__ == "__main__":
    engine = create_engine(DB_URI)

    for tbl in ("wpr_truth", "wpr_sliding"):
        inspect_wpr(tbl, engine)

In [6]:
from sqlalchemy import create_engine, text
import pandas as pd

# 📌 point to your combined DB
engine = create_engine("sqlite:///app/data/combined.db")

# 1) List all tables
with engine.connect() as conn:
    tables = pd.read_sql_query(
        text("SELECT name FROM sqlite_master WHERE type='table'"),
        conn
    )
print("Tables in DB:\n", tables["name"].tolist())

# 2) Fetch zero rows from bond_stocks to get its column names
with engine.connect() as conn:
    df_empty = pd.read_sql_query(
        text("SELECT * FROM bond_stocks LIMIT 0"),
        conn
    )
print("\nColumns in bond_stocks:\n", df_empty.columns.tolist())


Tables in DB:
 ['wpr_truth', 'wpr_sliding', 'daily_movement', 'daily_pipeline', 'pricing_vector', 'us_imports_exports', 'global_imports_exports', 'bond_stocks']

Columns in bond_stocks:
 ['date', 'VIX (Volatility)', 'NASDAQ Composite', 'Nikkei 225', 'OVX', 'S&P 500', 'T10Y3M Spread', 'T10Y2Y Spread', 'WTI Crude Oil']
